In [27]:
%matplotlib inline
# import libraries
import re
import numpy as np  # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt

num_epochs = 100
total_series_length = 50000
truncated_backprop_length = 15
state_size = 4
num_classes = 2
echo_step = 3
batch_size = 5
num_batches = total_series_length//batch_size//truncated_backprop_length

# find associated race/ethnicity charged article headlines and use sentiment detection
# to analyze positive and negatives
# https://www.kaggle.com/bittlingmayer/amazonreviews


Using TensorFlow backend.


In [178]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model


BASE_DIR = ''
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [182]:
# grab the data of 2014 news article headlines
news = pd.read_csv("../datasets/uci-news-aggregator.csv")

tokenizer = Tokenizer()
tokenizer.fit_on_texts(news['TITLE'])

# fit to a keras.sequences
sequences = tokenizer.texts_to_sequences(news['TITLE'])

In [183]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences)

Found 75286 unique tokens.


In [184]:
tokenizer.fit_on_texts(news['CATEGORY'])
label_sequences = tokenizer.texts_to_sequences(news['CATEGORY'])
labels = pad_sequences(label_sequences)

In [ ]:
# grab bias lexicon datasets, these occured twice from wiki edit corpus
bias_lex = pd.read_csv("../datasets/bias-lexicon/bias-lexicon.txt", sep="\n", header = None)

# other related lexicons // future features
# assertive_verbs = pd.read_csv("../datasets/bias_related_lexicons/assertives_hooper1975.txt", skiprows=7, sep="\n", header = None)
# factive_verbs = pd.read_csv("../datasets/bias_related_lexicons/factives_hooper1975.txt", skiprows=7, sep="\n", header = None)
# hedge_adverbs = pd.read_csv("../datasets/bias_related_lexicons/hedges_hyland2005.txt", skiprows=7, sep="\n", header = None)
# implicative_verbs = pd.read_csv("../datasets/bias_related_lexicons/implicatives_karttunen1971.txt", skiprows=6, sep="\n", header = None)
# report_verbs = pd.read_csv("../datasets/bias_related_lexicons/report_verbs.txt", skiprows=9, sep="\n", header = None)

In [203]:
#labels = keras.utils.to_categorical(np.asarray(bias_lex))

labels = to_categorical(labels)

print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

('Shape of data tensor:', (422419, 4036))
('Shape of label tensor:', (422419, 10))


In [211]:
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

In [212]:
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [213]:
embeddings_index = {}
f = open('../datasets/glove.6B/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [214]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [215]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=False)

In [217]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)  # global max pooling
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(10, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

# happy learning!
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=2, batch_size=128)

Train on 337936 samples, validate on 84483 samples
Epoch 1/2
337936/337936 [==============================] - 31410s - loss: 1.4011 - acc: 0.3597 - val_loss: 1.3130 - val_acc: 0.3618
Epoch 2/2
262400/337936 [======================>.......] - ETA: 15302s - loss: 1.3117 - acc: 0.3614

KeyboardInterrupt: 

In [154]:
# let's take a look at our data
bias_lex.head()

assertive_verbs.head()
factive_verbs.head()
hedges_adverbs.head()
implicative_verbs.head()
report_verbs.head()

# race based lexicons
ethnicity_lex =
race_lex =
country_lex = 

,0
0,accuse
1,acknowledge
2,add
3,admit
4,advise
